In [2]:
import numpy as np
import theano
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

In [ ]:
class RBM(object):
    """Restricted Boltzmann Machine (RBM)"""
    def __init__(
        self,
        input=None,
        n_visible=784,
        n_hidden=500,
        W=None,
        hbias=None,
        vbias=None,
        numpy_rng=None,
        theano_rng=None
    ):
        """
        RBM constructor
        """
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        
        if numpy_rng is None:
            numpy_rng = np.random.RandomState(213)
        
        if theano_rng is None:
            theano_rng = RandomStreams(numpy_rng.randint(2 ** 30))
        
        if W is None:
            initial_W = np.asarray(
                numpy_rng.uniform(
                    low=-4 * np.sqrt(6. / (n_hidden + n_visible)),
                    high=-4 * np.sqrt(6. / (n_hidden + n_visible))
                ),
                dtype=theano.config.floatX
            )
            W = theano.shared(value=initial_W, name='W', borrow=True)
        
        if hbias is None:
            hbias = theano.shared(
                value=np.zeros(
                    n_hidden,
                    dtype=theano.config.floatX
                ),
                name='hbias',
                borrow=True
            )
        
        if vbias is None:
            vbias = theano.shared(
                value=np.zeros(
                    n_visible,
                    dtype=theano.config.floatX
                ),
                name='vbias',
                borrow=True
            )
        
        self.input = input
        if not input:
            self.input = T.matrix('input')
        
        self.W = W
        self.hbias = hbias
        self.vbias = vbias
        self.theano_rng = theano_rng
        
        self.params = [self.W, self.hbias, self.vbias]
    
    def propup(self, vis):
        pre_sigmoid_activation = T.dot(vis, self.W) + self.hbias
        return [pre_sigmoid_activation, T.nnet.sigmoid(pre_sigmoid_activation)]
    
    def sample_h_given_v(self, v0_sample):
        pre_sigmoid_h1, h1_mean = self.propup(v0_sample)
        h1_sample = self.theano_rng.binomial(size=h1_mean.shape,
                                            n=1, p=h1_mean,
                                             dtype=theano.config.floatX
                                            )
        return [pre_sigmoid_h1, h1_mean, h1_sample]
    
    def propdown(self, hid):
        pre_sigmoid_activation = T.dot(hid, self.W.T) + self.vbias
        return [pre_sigmoid_activation, T.nnet.sigmoid(pre_sigmoid_activation)]
    
    def sample_v_given_h(self, h0_sample):
        pre_sigmoid_v1, v1_mean = self.propdown(h0_sample)
        v1_sample = self.theano_rng.binomial(size=v1_mean.shape,
                                             n=1, p=v1_mean,
                                             dtype=theano.config.floatX
                                            )
        return [pre_sigmoid_v1, v1_mean, v1_sample]
    
    def gibbs_hvh(self, h0_sample):
        pre_sigmoid_v1, v1_mean, v1_sample = self.sample_v_given_h(h0_sample)
        pre_sigmoid_h1, h1_mean, h1_sample = self.sample_h_given_v(v1_sample)
        return [pre_sigmoid_v1, v1_mean, v1_sample,
               pre_sigmoid_h1, h1_mean, h1_sample]
    
    def gibbs_vhv(self, v0_sample):
        pre_sigmoid_h1, h1_mean, h1_sample = self.sample_h_given_v(v0_sample)
        pre_sigmoid_v1, v1_mean, v1_sample = self.sample_v_given_h(h0_sample)
        return [pre_sigmoid_h1, h1_mean, h1_sample,
               pre_sigmoid_v1, v1_mean, v1_sample]
    
    def free_energy(self, v_sample):
        wx_b = T.dot(v_sample, self.W) + self.hbias
        vbias_term = T.dot(v_sample, self.vbias)
        hidden_term = T.sum(T.log(1 + T.exp(wx_b)), axis=1)
        return -hidden_term - vbias_term
    
    def get_cost_updates(self, lr=0.1, persistent=None, k=1):
        pre_sigmoid_ph, ph_mean, ph_sample = self.sample_h_given_v(self.input)
        
        if persistent is None:
            chain_start = ph_sample
        else:
            chain_start = persistent
            
        (
            [
                pre_sigmoid_nvs,
                nv_means,
                nv_samples,
                pre_sigmoid_nhs,
                nh_means,
                nh_samples
            ],
            updates
        ) = theano.scan(
                self.gibbs_hvh,
                outputs_info=[None, None, None, None, None, chain_start],
                n_steps=k
        )
        
        chain_end = nv_samples[-1]
        
        cost = T.mean(self.free_energy(self.input)) - T.mean(self.free_energy(chain_end))
        
        gparams = T.grad(cost, self.params, consider_constant=[chain_end])
        
        for gparam, param in zip(gparams, self.params):
            updates[param] = param - gparam * T.cast(
                lr,
                dtype=theano.config.floatX
            )
        
        if persistent:
            updates[persistent] = nh_samples[-1]
            monitor_cost = self.get_reconstruction_cost(updates,
                                                       pre_sigmoid_nvs[-1])
            
        return monitor_cost, updates

In [6]:
RandomStreams?

In [12]:
a = np.arange(12).reshape(3,4)
b = np.random.randn(3)
b.dot(a)

array([-1.93101144, -1.72587842, -1.52074541, -1.31561239])

In [13]:
update[3] = 2

NameError: name 'update' is not defined